<h1 style="color:#4E2A84;">Qubit Vise Demonstration</h1>

This notebook provides a demonstration of the *Qubit Vise* attack. The notebook shows how to create two attacker circuits with many $CNOT$ gates each and assign the circuits to specific physical qubits in the target quantum computer. It then demonstrates how to add a victim circuit and to assign it to qubits physically located between the two attacker circuits. The circuits are executed on the Rigetti Ankaa-3 quantum computer by using qBraid service. The example attacker circuits use many $CNOT$ gates to generates noise or crosstalk that affects the victim circuit. As of June 2025 this is first demonstration of two-sided crosstalk attack on a victim quantum circuit that is larger than 1 qubit, and also first type of this attack on Rigetti. The execution of the $CNOT$ gates in the attacker circuits on both sides of a victim causes crosstalk that is observed by analyzing how the victim's output statistics change without and with attack.

General overview of *Qubit Vise* attack is shown in the below figure. There are two sets of attackers, located on either side of the victim. As an example, in the figure there are two sets of attackers each with $10$ qubits, and victim also using $10$ qubits. This number of attacker and victim qubits is only used for illustrative purposes. The attackers and victim qubits are shown on the Rigetti Ankaa-3 topology.

<img src="images/qubit_vise_ankaa_3_overview.pdf" alt="Qubit Vise overview figure" style="width:600px;"/>

To demonstrate the crosstalk attack, this notebook leverages the `AWS` and `qBraid` cloud services, it uses the `Braket` development environment and targets a `Rigetti Ankaa-3` superconducting quantum computer.

In [ ]:
# reset all notebook variables just in case
%reset -f

<h1 style="color:#4E2A84;">Setup qBraid and Amazon Braket Environment</h1> 

This notebook assumes you are using qBraid to access quantum computers offered from Amazon Braket. This notebook assumes you have qBraid account active and have qBraid credits sufficient to exeucte the circuits. Generally, this notebook assumes you are working from [https://lab.qbraid.com].

To begin, make sure that you have installed the latest Amazon Braket environment in qBraid Lab ([installation instructions](https://docs.qbraid.com/lab/user-guide/environments#install-environment)) and have selected the `Python [Braket vX.X]` kernel. (You can switch kernels by going to the Menu Bar → Kernel → Change Kernel in [https://lab.qbraid.com]). 

You can vew the kernel currently selected in the top-right of the editor window: <img src="images/image_select_kernel.png" alt="Kernel selection" style="width:200px;"/>

<h1 style="color:#4E2A84;">General Setup for Coding</h1> 

Now, you need to import the appropriate libraries. This notebook uses Braket development enviroment, which is initialized with the below imports.

In [ ]:
# general imports
import matplotlib.pyplot as plt
import numpy as np

# magic command to display plots below cells that produce them
%matplotlib inline

# import Braket related modules
from braket.circuits import Circuit
from braket.aws import AwsDevice, AwsQuantumTask

<h1 style="color:#4E2A84;">Partial Code for Circuit with Many CNOT Gates</h1>

Below cells specify a circuit with many $CNOT$ gates. Note the qubits used by the $CNOT$ gates have specific indices. The indices, i.e. the physical qubits used in the circuit, were selected specifically based on the topology of the Rigetti Ankaa-3 quantum computer. The topology is exemplified later in this notebook. For now we simply show how to create a circuit with $CNOT$ gates, targetting $10$ qubits.

Note that in Braket, we assign the physical (hardware) qubits when we define our circuit. It is different from IBM Qiskit, where the mapping of circuit qubits to hardware qubits is handled during the compilation step.

Below you will define two circuits, each with different arrangament of $CNOT$ gates. After that, the two circuits can be concatenated together to create an even bigger circuit, which will be the actual circuit we will use.

In [ ]:
# define a circuit
circ_attacker_top_a = Circuit()

# add gates to the circuit
circ_attacker_top_a.cnot(77, 78)
circ_attacker_top_a.cnot(70, 71)
circ_attacker_top_a.cnot(63, 64)
circ_attacker_top_a.cnot(56, 57)
circ_attacker_top_a.cnot(49, 50)

In [ ]:
# print the circuit for inspecction
print(circ_attacker_top_a)

In [ ]:
# define a second circuit whith different arrangament of CNOT gates
circ_attacker_top_b = Circuit()

# add gates to the circuit
circ_attacker_top_b.cnot(78, 71)
circ_attacker_top_b.cnot(70, 63)
circ_attacker_top_b.cnot(64, 57)
circ_attacker_top_b.cnot(56, 49)

In [ ]:
# print the circuit for inspecction
print(circ_attacker_top_b)

<h1 style="color:#4E2A84;">Put Together Code for a Circuit with Many CNOT Gates</h1>

Now you will combine the smaller circuits, *circ_attacker_top_a* and *circ_attacker_top_b*, to create a bigger attacker circuit. You can adjust the *num_copies* variable to control how many copies of the $CNOT$ gate circuits will be concatenated together. The more copies, the more crosstalk will be generated. However, you need to pay attention that the attacker circuit is not longer than the victim (i.e. it does not have depth larger than the victim) as otherwise the output of the victim will be affected not just by the crosstalk, but also decoherence as the victim circuit qubits are idle as the attacker circuits still execute. Furthter note that none of the circuts' depth should be larger than the decoherence times of the target quantum computer being used.

In [ ]:
# create the main circuit
circ_attcker_top = Circuit()

# set now many copies of the circuits to concatenate together
num_copies = 2

# use a loop to concatenate many copies of the circuits together
for n in range(0,num_copies):
    circ_attcker_top += circ_attacker_top_a
    circ_attcker_top += circ_attacker_top_b

In [ ]:
# print the circuit
print(circ_attcker_top)

<h1 style="color:#4E2A84;">Second Circuit with Many CNOT Gates</h1>

Below you will create a second attacker circuit, which will be physically placed below the victim. Similar to the first attacker circuit, it is composed of alternating sets of $CNOT$ gates. The physical qubits are assigned based on the Rigetti Ankaa-3 quantum computer's topology.

In [ ]:
# define a circuit
circ_attacker_bottom_a = Circuit()

# add gates to the circuit
circ_attacker_bottom_a.cnot(81, 82)
circ_attacker_bottom_a.cnot(74, 75)
circ_attacker_bottom_a.cnot(67, 68)
#circ_attacker_bottom_a.cnot(60, 62) #remove CNOT due to compilation error
circ_attacker_bottom_a.cnot(53, 54)

In [ ]:
# define a second circuit whith different arrangament of CNOT gates
circ_attacker_bottom_b = Circuit()

# add gates to the circuit
circ_attacker_bottom_b.cnot(81, 74)
circ_attacker_bottom_b.cnot(75, 68)
circ_attacker_bottom_b.cnot(67, 60)
#circ_attacker_bottom_b.cnot(62, 54) #remove CNOT due to compilation error

In [ ]:
# create the main circuit
circ_attcker_bottom = Circuit()

# set now many copies of the circuits to concatenate together
num_copies = 2

# use a loop to concatenate many copies of the circuits together
for n in range(0,num_copies):
    circ_attcker_bottom += circ_attacker_bottom_a
    circ_attcker_bottom += circ_attacker_bottom_b

<h1 style="color:#4E2A84;">Create the Qubit Vise Circuit</h1>

Now you will combine the two attacker circuits to create the *Qbit Vise* attacker circuit. Below figure shows an illustration of where the attacker qubits are located.

<img src="images/qubit_vise_ankaa_3_attackers.pdf" alt="Layout of two attacker circuits" style="width:600px;"/>

In [ ]:
# create the qubit vise circuit
circ_qubit_vise = Circuit()

# concatenate the two sets of CNOT gate circuits
circ_qubit_vise += circ_attcker_top
circ_qubit_vise += circ_attcker_bottom

# keep track of qubits used, later needed for adding measurement gates
qubits_used = []
qubits_used.extend([77, 78, 70, 71, 63, 64, 56, 57, 49, 50])
qubits_used.extend([81, 82, 74, 75, 67, 68, 60, 62, 53, 54])

# print the circuit for visual inspection
print("Qubits used:", qubits_used)
print("Gate depth:", circ_qubit_vise.depth)
print(circ_qubit_vise)

<h1 style="color:#4E2A84;">Define a Victim Circuit</h1>

Bellow you will define a victim circuit, one that should do useful computation so that later its output can be compared without and with attack. Note, two copies of the victim circuit are needed, one is placed inside the *Qubit Vise* to analyze effect of cross talk, and second outside, ideally far away. To compare the potential results of the crosstalk to behavior of an unaffected qubit, you need this second copy as a reference somewhere else in the topology. Below example assigns qubits in the bottom-right of the Rigetti Ankaa-3 machines as the location for the reference circui: it is far away from the qubits where $CNOT$ gates execute, and so it should not be impacted by the crosstalk.

<h2 style="color:#4E2A84;">Bell Circuit</h2>

Use next two cells for testing Bell circuit, otherwise skip them.

In [ ]:
# create the victim bell circuit, 2 qubit
circ_victim_main = Circuit()

circ_victim_main.h(72)
circ_victim_main.cnot(72, 65)

# keep track of qubits used, later needed for adding measurement gates
qubits_used.extend([72, 65])

In [ ]:
# create the reference bell victim circuit, 2 qubit
circ_victim_reference = Circuit()

circ_victim_reference.h(12)
circ_victim_reference.cnot(12, 5)

# keep track of qubits used, later needed for adding measurement gates
qubits_used.extend([12, 5])

<h2 style="color:#4E2A84;">Ising Circuit</h2>

Use next two cells for testing Ising circuit, otherwise skip them.

In [ ]:
# create the victim ising circuit, 4 qubit

# qubit map
#qubit_map = {0: 79, 1: 80, 2: 72, 3: 73, 4: 65, 5: 66}
qubit_map = {0: 79, 1: 80, 2: 73, 3: 72}
             
# parameters
n_qubits = 4
J = 1.0
h = 0.8
gamma = np.pi / 4  # Cost unitary angle
beta = np.pi / 8   # Mixer unitary angle

# create the circuit
circ_victim_main = Circuit()

# step 1: Hadamard gates (initialize to |+>)
for q in range(n_qubits):
    circ_victim_main.h(qubit_map[q])

# step 2: Apply ZZ interactions (cost unitary)
for i in range(n_qubits - 1):  # 0 to 2
    circ_victim_main.cz(qubit_map[i], qubit_map[i+1])
    circ_victim_main.rz(qubit_map[i+1], -2 * gamma * J)
    circ_victim_main.cz(qubit_map[i], qubit_map[i+1])

# step 3: Apply RX mixer terms
for q in range(n_qubits):
    circ_victim_main.rx(qubit_map[q], 2 * beta * h)

sorted_values = [v for k, v in sorted(qubit_map.items())]
qubits_used.extend(sorted_values)
print(qubits_used)
print(circ_victim_main)

In [ ]:
# create the reference ising circuit, 4 qubit

# qubit map
#qubit_map = {0: 19, 1: 20, 2: 12, 3: 13, 4: 5, 5: 6}
qubit_map = {0: 19, 1: 20, 2: 13, 3: 12}
             
# parameters
n_qubits = 4
J = 1.0
h = 0.8
gamma = np.pi / 4  # Cost unitary angle
beta = np.pi / 8   # Mixer unitary angle

# create the circuit
circ_victim_reference = Circuit()

# step 1: Hadamard gates (initialize to |+>)
for q in range(n_qubits):
    circ_victim_reference.h(qubit_map[q])

# step 2: Apply ZZ interactions (cost unitary)
for i in range(n_qubits - 1):  # 0 to 4
    circ_victim_reference.cz(qubit_map[i], qubit_map[i+1])
    circ_victim_reference.rz(qubit_map[i+1], -2 * gamma * J)
    circ_victim_reference.cz(qubit_map[i], qubit_map[i+1])

# step 3: Apply RX mixer terms
for q in range(n_qubits):
    circ_victim_reference.rx(qubit_map[q], 2 * beta * h)

sorted_values = [v for k, v in sorted(qubit_map.items())]
qubits_used.extend(sorted_values)
print(qubits_used)
#print(circ_victim_reference)

<h2 style="color:#4E2A84;">GHZ Circuit</h2>

Use next two cells for testing GHZ circuit, otherwise skip them.

In [ ]:
# create the victim ghz circuit, 6 qubit

# qubit map per topology
qubit_map = {0: 65, 1: 72, 2: 79, 3: 66, 4: 73, 5: 58}

# make new circuit
circ_victim_main = Circuit()

# put qubit 0 into superposition
circ_victim_main.h(qubit_map[0])

# chain of CNOTs to entangle all qubits with qubit 0, add swap gates as needed per topology
circ_victim_main.cnot(qubit_map[0], qubit_map[1]) # cnot with qubit 1
circ_victim_main.swap(qubit_map[1], qubit_map[2])
circ_victim_main.cnot(qubit_map[0], qubit_map[1]) # cnot with qubit 2
circ_victim_main.cnot(qubit_map[0], qubit_map[3]) # cnot with qubit 3
circ_victim_main.swap(qubit_map[3], qubit_map[4])
circ_victim_main.cnot(qubit_map[0], qubit_map[3]) # cnot with qubit 4
circ_victim_main.cnot(qubit_map[0], qubit_map[5]) # cnot with qubit 5

sorted_values = [v for k, v in sorted(qubit_map.items())]
qubits_used.extend(sorted_values)
print(qubits_used)
#print(circ_victim_main)

In [ ]:
# create the victim ghz circuit, 6 qubit

# qubit map per topology
qubit_map = {0: 12, 1: 19, 2: 26, 3: 13, 4: 20, 5: 5}

# make new circuit
circ_victim_reference = Circuit()

# put qubit 0 into superposition
circ_victim_reference.h(qubit_map[0])

# chain of CNOTs to entangle all qubits with qubit 0, add swap gates as needed per topology
circ_victim_reference.cnot(qubit_map[0], qubit_map[1]) # cnot with qubit 1
circ_victim_reference.swap(qubit_map[1], qubit_map[2])
circ_victim_reference.cnot(qubit_map[0], qubit_map[1]) # cnot with qubit 2
circ_victim_reference.cnot(qubit_map[0], qubit_map[3]) # cnot with qubit 3
circ_victim_reference.swap(qubit_map[3], qubit_map[4])
circ_victim_reference.cnot(qubit_map[0], qubit_map[3]) # cnot with qubit 4
circ_victim_reference.cnot(qubit_map[0], qubit_map[5]) # cnot with qubit 5

sorted_values = [v for k, v in sorted(qubit_map.items())]
qubits_used.extend(sorted_values)
print(qubits_used)
#print(circ_victim_reference)

<h1 style="color:#4E2A84;">Combined Circuit with Attackers and Victim</h1>

You will now combine all the circuits together. The attackers, the victim, and the second reference victim are shown in the figure below:

<img src="images/qubit_vise_ankaa_3_victim.pdf" alt="Qubit layout with attackers and victim circuits" style="width:600px;"/>

In [ ]:
# create the combined circuit
circ = Circuit()

# put all the circuits together
circ += circ_qubit_vise
circ += circ_victim_main
circ += circ_victim_reference

In [ ]:
# print circuit for visual inspection
#print(circ)

<h1 style="color:#4E2A84;">Add Measurement to All Qubits</h1>

With the circuit completed, the last part is now to add measurement to all the qubits.

In [ ]:
# add final measurement to all qubits
for q in qubits_used:
    circ.measure(q)

In [ ]:
# print circuit for inspection
#print(circ)

<h1 style="color:#4E2A84;">Estimate the Circuit Duration</h1>

As a minor sanity check, we want to check the duration of the circuit we generated. It is important to note that the qubits decohere, and computation cannot usually run longer than the so called $T1$ time ($T1$ time is an average, so in some cases compuation can run slightly longer than $T1$). If the circuit with many $CNOT$ gates were used to generate crosstalk and evaluate potentail qubit flits, the circuit with many $CNOT$ gates has to execute in the time shorter than $T1$. Otherwise, any qubit flips or state changes in other qubits affected by the crosstalk may actually be due to decoherence and not crosstalk.

In summary, when running any experiments or circuits that are used to generate and evaluate crosstalk, all the circuits have to finish execution in time less than $T1$ so that the qubit flips can be attribtued to crosstalk.

In [ ]:
# circuit to estimate
circ_to_estimate = circ

# typical approximate parameters for Ankaa-3, time units are ns
typical_parameters_ankaa_3_ns = {
    "t1": 22000,
    "t2": 19000,
    "cnot": 72,
}

# get depth of the circuit
print("Gate depth:", circ_to_estimate.depth)

# sanity check
if circ_to_estimate.depth * typical_parameters_ankaa_3_ns['cnot'] < typical_parameters_ankaa_3_ns['t1']:
    print("Based on gate depth, total circuit execution time seems to be less than the decoherence time.")
else:
    print("Based on gate depth, total circuit execution time seems to be more than the decoherence time.")
    print("Be careful that any experiment results are affected by decoherence!")


<h1 style="color:#4E2A84;">Skip Simulation</h1> 

Simple state-vector simulators do not simulate crosstalk, thus running circuits used for generating crosstalk, or trying to evalute crosstalk in state-vector simulators is not helpful. You will directly proceed to estimate the execution cost and run the circuit on real quantum computer.

<h1 style="color:#4E2A84;">Set Number of Shots</h1> 

Before estimating the cost and executing the circuit, we need to specify the number of shots. As usually, a number aroung $1000$ should be sufficient.

In [ ]:
# set number of shots
circuit_shots = 1000

<h1 style="color:#4E2A84;">Estimate qBraids Credits Needed</h1> 

The cost to run a circuit on a quantum computer is divided into per-task cost (effectively the cost to submit a circuit for execution) and per-shot cost (additional cost for each shot).

You can view the cost by selecting $DEVICES$ on the right-hand side of the editor, then search for Rigetti Ankaa-3 (AWS) and finally selecting the pricing cell. <img src="images/image_pricing.png" alt="View pricing" style="width:300px;"/>

The current cost is $30$ qBraid Credits per-task on Ankaa-3 and then $0.09$ qBraid Credits per-task.

Currently each qBraid credit is valued at $0.01$ USD. This means that $100$ credits equate to $1.00$ USD.

In [ ]:
# compute cost
perTaksCost = 30.00
perShotCost = 0.09
totalCost = perTaksCost+perShotCost*circuit_shots
totalCostDollars = totalCost/100

# print cost
print(f'Cost to run Bell circuit on Rigetti Ankaa-3 via AWS is:')
print(f'{totalCost} qBraid Credit or equivalently ${totalCostDollars} USD')

<h1 style="color:#4E2A84;">Purchase or Request qBraid Credits</h1> 

You will need qBraid Credits to run the code on actual quantum computer. You can purchase the qBraid credits, or request them from your organization.

<h1 style="color:#4E2A84;">Submit the Circuit to Rigetti Ankaa-3</h1>

You can now submit the circuit for execution.

Please note, we’ve disabled qubit rewiring to allow us to ensure the qubits numbers we used in the circuit definition are exactly the ones which be used to execute our circuit. For Rigetti, qubit rewiring must be turned off by setting `disableQubitRewiring=True` for use with verbatim compilation. If `disableQubitRewiring=False` is set when using verbatim boxes in a compilation, the quantum circuit fails validation and does not run.

In [ ]:
# set up device
ankaa = AwsDevice('arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3')

# run circuit
ankaa_task = ankaa.run(circ, shots=circuit_shots, disable_qubit_rewiring=True)
    
# get id and status of submitted task
ankaa_task_id = ankaa_task.id
ankaa_status = ankaa_task.state()

# print task status
print(f'Status of task: {ankaa_status}\n')

# print task id for future reference
print(f'Task id: {ankaa_task_id}')

<h1 style="color:#4E2A84;">Wait for Circuit to Complete</h1>

You can use the task id to check if the circuit has completed. The results cannot be accessed until the circuit has completed and you may have to run the next cell multiple times until you see the status of the circuit as completed.

In [ ]:
# get task arn
ankaa_task_arn = ''

# load the quantum task
ankaa_task_id = AwsQuantumTask(arn=ankaa_task_arn)

# print status
status = ankaa_task_id.state()
print("Status of the task:", status)
#print(ankaa_task_id.state())
#print(ankaa_task_id.metadata())

<h1 style="color:#4E2A84;">Load Results</h1>

When status of a taks is $COMPLETED$ you can access and analyze the results. Below cell simply gets results from the circuit. Cells further below are used to analyze specific victim circuits.

In [ ]:
# get task arn
#ankaa_task_arn = ''

# load the quantum task
ankaa_task_id = AwsQuantumTask(arn=ankaa_task_arn)

# get results
ankaa_results = ankaa_task_id.result()

# get order of qubits in the measurement counts
print('Order of qubits in the measurement counts:',ankaa_results.measured_qubits)

# get counts
counts = ankaa_results.measurement_counts
#print('Measurement counts:', counts)

<h1 style="color:#4E2A84;">Victim Circuit Analysis</h1>

Use the below cells to analyze the victim circuit. Generally, the least significant bits in the measuremet bitstrings represent the reference outputs, while the next set of bits right above the least significant bits represents the victim circuit outputs under attack.

In [ ]:
# helper function to check if bits match a pattern
def is_bit_pattern_set(binary_str, meas_map, pattern_str):
    reversed_str = binary_str[::-1]
    test_str = str()
    for key in sorted(meas_map):
        test_str += reversed_str[meas_map[key]]
    return test_str == pattern_str

# collect counts for each possible state of the circuit
def get_counts(counts, num_qubits, meas_map):
    temp_counts = dict()
    for j in range(2**num_qubits):
        binary_str = bin(j)[2:].zfill(num_qubits)
        temp_counts[binary_str] = 0

    for j in range(2**num_qubits):
        for key in counts.keys():
            binary_str = bin(j)[2:].zfill(num_qubits)
            if is_bit_pattern_set(key, meas_map, binary_str):
                temp_counts[binary_str] = temp_counts[binary_str] + counts[key]

    print(temp_counts)
    return temp_counts

test_circ = 'ising' # bell, ising, ghz

if test_circ == 'bell':
    # bell state circuit
    num_qubits = 2
    meas_map_ref = {0: 1, 1: 0}
    meas_map_test = {0: 3, 1: 2}
elif test_circ == 'ising':
    # isinig 4 qubit circuit
    num_qubits = 4
    meas_map_ref = {0: 3, 1: 2, 2: 1, 3: 0}
    meas_map_test = {0: 7, 1: 6, 2: 5, 3: 4}
else:
    # ghz 6 qubit circuit
    num_qubits = 6
    meas_map_ref = {0: 5, 1: 4, 2: 3, 3: 2, 4: 1, 5: 0}
    meas_map_test = {0: 11, 1: 10, 2: 9, 3: 8, 4: 7, 5: 6}

# get counts for the circuit
counts_no_attack = get_counts(counts, num_qubits, meas_map_ref)

# get counts for the circuit
counts_with_attack = get_counts(counts, num_qubits, meas_map_test)

In [ ]:
# plot counts without and with attack
if test_circ == 'ghz':
    plt.figure(figsize=(10, 3))
else:
    plt.figure(figsize=(5, 3))
plt.bar(counts_no_attack.keys(), counts_no_attack.values(), color='mediumslateblue')
plt.xticks(rotation=90)
plt.xlabel('Bitstring')
plt.ylabel('Counts')
plt.title('Reference Measurement Outcomes')
plt.grid(axis='y', linestyle='--', alpha=0.6)
filename = test_circ + '_ref' + '.pdf'
plt.savefig(filename, bbox_inches='tight')
plt.show()

if test_circ == 'ghz':
    plt.figure(figsize=(10, 3))
else:
    plt.figure(figsize=(5, 3))
plt.bar(counts_with_attack.keys(), counts_with_attack.values(), color='mediumslateblue')
plt.xticks(rotation=90)
plt.xlabel('Bitstring')
plt.ylabel('Counts')
plt.title('Measurement Outcomes Under Attack')
plt.grid(axis='y', linestyle='--', alpha=0.6)
filename = test_circ + '_test' + '.pdf'
plt.savefig(filename, bbox_inches='tight')
plt.show()

In [ ]:
# compute variational distance
def variational_distance(counts1, counts2):
    # Normalize both dictionaries
    total1 = sum(counts1.values())
    total2 = sum(counts2.values())
    probs1 = {k: v / total1 for k, v in counts1.items()}
    probs2 = {k: v / total2 for k, v in counts2.items()}

    # Get union of all keys
    all_keys = set(probs1.keys()) | set(probs2.keys())

    # Compute variational distance
    distance = 0.5 * sum(abs(probs1.get(k, 0) - probs2.get(k, 0)) for k in all_keys)
    return distance

tvd = variational_distance(counts_with_attack, counts_no_attack)
print('Total variational distance:', round(tvd, 5))